# EE447: Full State Feedback

Eric Klavins

Copyright &copy; University of Washington, 2019

# Code

In [3]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sympy import *
from matplotlib import animation

# Uncomment on Google colab
# !pip install JSAnimation
# !pip install control

from JSAnimation.IPython_display import display_animation

%matplotlib inline

# Comment out in Google colab 
init_printing(use_latex='mathjax')

# Uncomment in below Google colab to render sympy equations nicely
# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
#     javascript(url=url)
#     return printing.latex(exp,**options)

# Full State Feedback in Phase Canonical Form

Say

$$
G(s) = \frac{b_3 s^3 + b_2 s^2 + b_1 s + b_0}
            {s^4 + a_3 s^3 + a_2 s^2 + a_1 s + a_0}
$$

In phase canonical form this is 

\begin{align}
\dot \x & = \begin{pmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
-a_0 & -a_1 & -a_2 & -a_3 
\end{pmatrix} \x + \begin{pmatrix}
0 \\
0 \\
0 \\
1
\end{pmatrix} u \\
y & = ( b_0 \; b_1 \; b_2 \; b_3 ) \; \x .
\end{align}

For now, we will ignore $y$ an <span style="background: yellow">suppose we have direct measurements of $\x$ </span>.

by construction, the characteristic equation (denominator of the transfer function) can be read off of the matrix by construction, and is 

$$
s^4 + a_3 s^3 + a_2 s^2 + a_1 s^1 + a_0 .
$$

**Full state feedback** means setting 

\begin{align}
u & = -K \x + r \\
  & = - (k_0 \; k_1 \; k_2 \; k_3) \; \x + r
\end{align}

where the $K$ is a matrix of gains *that you get to design* and $r$ is a new *reference input*. 

In blue we show what we have added to the system:

<img width=55% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/fsf.png">

Substituting in this value of $u$ back into the above we get

\begin{align}
\dot x & = A \x + B(-K \x + r ) \\
       & = (A-BK) \x + Br
\end{align}

and (still using phase variables) we get

\begin{align}
A-BK & = \begin{pmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
-a_0 & -a_1 & -a_2 & -a_3 
\end{pmatrix} - \begin{pmatrix}
0 \\
0 \\
0 \\
1
\end{pmatrix} (K_0 \; K_1 \; K_2 \; K_3) \\
 & = \begin{pmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
-a_0-k_0 & -a_1-k_1 & -a_2-k_2 & -a_3-k_3 
\end{pmatrix} 
\end{align}

which has characteristic polynomial

$$
s^4 + (a_3+k_3) s^3 + (a_2+k_2) s^2 + (a_1+k_1) s^1 + (a_0 +k_0)
$$

What's important is that by choosing $K$, we can set the coefficients of the new characteristic polynomial to be whatever we want them to be, and therefore we can place the poles wherever we want them to go!

Example:
---

Say

$$
G(s) = \frac{1}{s^2+s}
$$

with phase canonical form

$$
\dot \x = \begin{pmatrix} 
  0 & 1 \\
  0 & -1
\end{pmatrix} \x + \begin{pmatrix}
0 \\
1
\end{pmatrix} u
$$ 

$$
y = ( 1 \; 0 ) \; \x
$$

Suppose we want to design a controller so that the closed loop poles are at

$$
-1 \pm i.
$$

**Solution**: We want the closed loop characteristic polynomial to be

$$
s^2 + 2s + 2
$$

Setting $u = -K \x + r$ gives the new characteristic polynomial

$$
s^2 + (a_1+k_1)s + (a_0+k_0) \; = \; s^2 + (1+k_1)s + k_0
$$

setting the coefficients equal to the desired coefficients gives the two equations

\begin{align}
k_1 + 1 & = 2  \\
k_0 & = 2
\end{align}

so that a choice of $k_1 = 1$ and $k_0 = 2$ will put the closed loop poles at the desired locations.

# Arbitrary States

Example:
---

Design a full state feedback controller for the system

$$ 
\dot \x = \begin{pmatrix}
1 & 2 & 1 \\
0 & -1 & 0 \\
0 & 0 & 1
\end{pmatrix} \x + \begin{pmatrix}
0 \\
1 \\
1
\end{pmatrix} u
$$

that has 10% overshoot and a settling time of 2s, using the 2nd order pole approximation.

**Solution:** Translating into second order pole locations, we want

$$
s = -2 \pm 2.7 j
$$

for the dominant poles. Since the system is 3rd order, there is one more pole. Let's put it at $-5$. 

The resulting desired polynomial is

In [24]:
var("s")
desired_charpoly = ((s+5)*(s+2-2.7*I)*(s+2+2.7*I)).expand()
desired_charpoly

 3      2                  
s  + 9⋅s  + 31.29⋅s + 56.45

Note that to determine the characteristic polynomial of $A-BK$ we need to find $(sI-A+BK)^{-1}$, which has the determinant of $sI-A+BK$ in the denominator.

In [44]:
var("k0 k1 k2")
A = Matrix([
    [1,2,1],
    [0,-1,0],
    [0,0,1]
])
B = Matrix([
    [0],
    [1],
    [1]
])
K = Matrix([[k0,k1,k2]])
charpoly = (s*eye(3)-(A-B*K)).det()
charpoly

                  2                     2         3    2        
3⋅k₀⋅s - k₀ + k₁⋅s  - 2⋅k₁⋅s + k₁ + k₂⋅s  - k₂ + s  - s  - s + 1

Now we set the coefficients of the two polynomials equal and solve for the gains.

In [46]:
c1 = Poly(desired_charpoly,s).coeffs()
c2 = Poly(charpoly,s).coeffs()
Matrix(c1), Matrix(c2)

⎛⎡ 1.0 ⎤, ⎡        1        ⎤⎞
⎜⎢     ⎥  ⎢                 ⎥⎟
⎜⎢ 9.0 ⎥  ⎢   k₁ + k₂ - 1   ⎥⎟
⎜⎢     ⎥  ⎢                 ⎥⎟
⎜⎢31.29⎥  ⎢ 3⋅k₀ - 2⋅k₁ - 1 ⎥⎟
⎜⎢     ⎥  ⎢                 ⎥⎟
⎝⎣56.45⎦  ⎣-k₀ + k₁ - k₂ + 1⎦⎠

In [49]:
gains = solve(Matrix(c1)-Matrix(c2),[k0,k1,k2])
K.subs(gains)

[48.87  57.16  -47.16]

In [51]:
(A-B*K.subs(gains)).eigenvals()

⎧            27⋅ⅈ          27⋅ⅈ   ⎫
⎨-5: 1, -2 - ────: 1, -2 + ────: 1⎬
⎩             10            10    ⎭

# Integral Control

You can also do integral control, in cases where you want to ensure you can track a step response with no steady state error. We just need to add a new state for the integrator:

\begin{align}
\dot x_I & = r - y \\
         & = r - C \x
\end{align}

Together with $\dot \x = A \x + B u$ and $y = C \x$ we get the $n+1$ dimensional system

\begin{align}
\begin{pmatrix}
\dot \x \\
\dot x_I
\end{pmatrix} & = \begin{pmatrix}
A & 0 \\
-C & 0 
\end{pmatrix} \begin{pmatrix}
\x \\
x_I
\end{pmatrix} + \begin{pmatrix}
B \\
0
\end{pmatrix} u + \begin{pmatrix}
0 \\
1
\end{pmatrix} r
\end{align}

Putting

\begin{align}
u & = -K \x + k_i x_I \\
  & = - (K \;\; -k_I)  \begin{pmatrix}
\x \\
x_I
\end{pmatrix}
\end{align}

Substituting this definition of $u$ into the above equations for $\dot \x$ gives

\begin{align}
\begin{pmatrix}
\dot \x \\
\dot x_I
\end{pmatrix} & = \begin{pmatrix}
A-BK & B k_I \\
-C & 0 
\end{pmatrix} \begin{pmatrix}
\x \\
x_I
\end{pmatrix}  + \begin{pmatrix}
0 \\
1
\end{pmatrix} r.
\end{align}

In block diagram form, this is

<img width=75% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/fsf-integral.png">


Example
---

Say

\begin{align}
\dot \x & = \begin{pmatrix}
  2 & 1 \\
  0 & -1
\end{pmatrix} \x + \begin{pmatrix}
0 \\
1
\end{pmatrix} u \\
y & = ( 1 \; 2 ) \; \x
\end{align}

Adding an integrator and controller givies

$$
A-BK =  \begin{pmatrix}
2 & 1 \\
-k_0 & -k_1 - 1
\end{pmatrix}
$$

and

$$
Bk_I = \begin{pmatrix}
0 \\
k_I
\end{pmatrix} 
$$

so

\begin{align}
\begin{pmatrix}
\dot x_1 \\
\dot x_2 \\
\dot x_I 
\end{pmatrix} & = \begin{pmatrix}
  2 & 1 & 0 \\
  -k_0 & -k_1-1 & k_I \\
  -1 & -2 & 0
\end{pmatrix} \begin{pmatrix}
x_1 \\
x_2 \\
x_I 
\end{pmatrix} + \begin{pmatrix}
0 \\
0 \\
1
\end{pmatrix} r \\
y & = ( 1 \; 2 \; 0 ) \; \begin{pmatrix}
x_1 \\
x_2 \\
x_I 
\end{pmatrix} .
\end{align} 

We can find the characteristic equation to see if we can set the poles using sympy:

In [6]:
var("s k0 k1 ki")
A = Matrix([
    [2,1,0],
    [-k0,-k1-1,ki],
    [-1, -2, 0]
])
poly = (s*eye(3)-A).det().collect(s)
poly

         3    2                                    
-3⋅ki + s  + s ⋅(k₁ - 1) + s⋅(k₀ - 2⋅k₁ + 2⋅ki - 2)

You can see from the above that the three coefficients have different combinations of the gains and we can choose them to get any desired coefficients we want. For example, 
to get

$$
s^3+ 3s^2 + s^2 + 1
$$

we do the following

In [7]:
solve(
    Matrix([Poly(poly,s).coeffs()]) 
    - Matrix([[1,3,3,1]])
,[k0,k1,ki])

{k₀: 41/3, k₁: 4, ki: -1/3}

# What could go wrong?

Consider the system

\begin{align}
\dot \x & = \begin{pmatrix}
  -1 & 0 \\
  0 & -2
\end{pmatrix} \x + \begin{pmatrix} 
0 \\
1
\end{pmatrix} u \\
y & = ( 0 \; 1 ) \; \x
\end{align}

Note that the input $u$ does not affect $\x$ in any way. When we try to design a full state feedback controller, we get

\begin{align}
A - BK & = \begin{pmatrix}
  -1 & 0 \\
  0 & -2
\end{pmatrix} - \begin{pmatrix}
0 \\
1
\end{pmatrix} ( k_0 \;\; k_1 ) \\
  &= \begin{pmatrix}
  -1 & 0 \\
  -k_0 & -k_1
\end{pmatrix}
\end{align}

Taking the determinant $|s I - A + BK|$ gives

\begin{align}
\left |
\begin{pmatrix}
s+1 & 0 \\
k_0 & s+k_1
\end{pmatrix}
\right | & = (s+1)(s+k_1) \\
         & = s^2 + (k_1+1)s + k_1
\end{align}

The gain $k_0$ is not present and $k_1$ can not be chosen to give any coefficients you want, but rather only along a one dimensional curve in the $s$ plane parameterized by $k_1$. Therefore,

> Full state feedback only works for certain combinations of $A$ and $B$ matrices. 